In [4]:
df = None
with ClusterConnection(config_file="utils/cluster_config.json") as connection:
    file_manager = FileManager(connection)
    job_manager = JobManager(connection, file_manager, SlurmManager())
    flux_manager = FluxCorrelationIndicator(connection, file_manager, job_manager)

    # Define the job name and execute the flux
    job_name = "heliumcisd22631g"
    flux_manager.handle_flux(job_name, molecule, method, basis)
    df = file_manager.get_results(job_name)


Connected to atlas-edr.sw.ehu.es.
Gaussian input file './test/heliumcisd22631g.com' generated successfully using the 'markel' combination.
Command output: 
Uploaded test/heliumcisd22631g.com to /dipc/javidom/proyect-3-indicator/heliumcisd22631g/heliumcisd22631g.com on the cluster.
Uploaded slurm_scripts/heliumcisd22631g.slurm to /dipc/javidom/proyect-3-indicator/heliumcisd22631g/heliumcisd22631g.slurm on the cluster.
Command output: 
Command output: 
Command output: 
Command output: 
Command output: 
Command output: Submitted batch job 2157840

Command output:                JOBID   PARTITION         QOS                  NAME          USER    ST         TIME  NODES  NODELIST(REASON)
             2157840     general     regular      heliumcisd22631g       javidom    PD         0:00      1  (Priority)

Command output:                JOBID   PARTITION         QOS                  NAME          USER    ST         TIME  NODES  NODELIST(REASON)
             2157840     general     regular   

In [5]:
print(df)

           x    y     z   on_top  density
0        0.0  0.0  0.00  3.19709  2.94244
1        0.0  0.0  0.05  2.89811  2.79296
2        0.0  0.0  0.10  2.22100  2.42447
3        0.0  0.0  0.15  1.54428  1.99772
4        0.0  0.0  0.20  1.05023  1.62621
...      ...  ...   ...      ...      ...
1030296  5.0  5.0  4.80  0.00000  0.00000
1030297  5.0  5.0  4.85  0.00000  0.00000
1030298  5.0  5.0  4.90  0.00000  0.00000
1030299  5.0  5.0  4.95  0.00000  0.00000
1030300  5.0  5.0  5.00  0.00000  0.00000

[1030301 rows x 5 columns]


In [17]:
def obtain_atomic_indicator(df):
        # Filter data where y and z are 0
    filtered_df = df[(df['z'] <= 0.000001)].copy()
    
    # Create indicator column
    filtered_df['indicator'] = 2* filtered_df['on_top'] / (filtered_df['density'] ) ** 2
    
    # Select relevant columns
    result_df = filtered_df[['x', 'y','indicator']].dropna(subset=['indicator'])

    return result_df 

In [18]:
df.keys()

Index(['x', 'y', 'z', 'on_top', 'density'], dtype='object')

In [19]:
print(obtain_atomic_indicator(df))

           x     y  indicator
0       0.00  0.00   1.477065
101     0.00  0.05   1.486091
202     0.00  0.10   1.511384
303     0.00  0.15   1.547807
404     0.00  0.20   1.588514
...      ...   ...        ...
724675  3.55  0.20   0.815534
724776  3.55  0.25   0.815536
724877  3.55  0.30   0.815525
724978  3.55  0.35   0.815530
725079  3.55  0.40   0.815526

[4085 rows x 3 columns]


In [25]:
def plot_comparison(molecule, calculated_indicator, limit):
    """
    Displays a side-by-side comparison of the reference image and calculated contour and line plots for a given atom.
    Automatically determines the largest square region for the grid.

    Parameters:
    - molecule (str): Name of the atom (e.g., 'He', 'Ne', 'Ar').
    - calculated_indicator (pd.DataFrame): DataFrame containing calculated OTR values with 'x', 'y', and 'indicator' columns.
    """
    
    # Load the reference image
    img_path = f'./notebooks/figures/{molecule.lower()}.png'
    try:
        reference_img = Image.open(img_path)
    except FileNotFoundError:
        print(f"Reference image for {molecule} not found at {img_path}. Placeholder will be shown.")
        reference_img = None
    
    # Sort unique x and y values
    unique_x = np.sort(calculated_indicator['x'].unique())
    unique_y = np.sort(calculated_indicator['y'].unique())
    
    # Find the largest `q` for a square grid within [0, q]
    max_q = min(unique_x[-1], unique_y[-1])
    for q in reversed(unique_x):  # Start from the largest possible q and move down
        filtered_x = unique_x[unique_x <= q]
        filtered_y = unique_y[unique_y <= q]
        
        if len(filtered_x) == len(filtered_y):
            break  # Largest q that forms a square grid is found

    # Filter data to only include values within [0, q]
    filtered_data = calculated_indicator[(calculated_indicator['x'] <= limit) & (calculated_indicator['y'] <= limit)]
    
    # Update unique values for x and y based on filtered data
    unique_x = np.sort(filtered_data['x'].unique())
    unique_y = np.sort(filtered_data['y'].unique())
    num_x = len(unique_x)
    num_y = len(unique_y)
    
    # Reshape data for contour plot
    try:
        # Create meshgrid for x and y values
        x_vals, y_vals = np.meshgrid(unique_x, unique_y, indexing='ij')
        indicator_vals = filtered_data['indicator'].values.reshape((num_x, num_y))
    except ValueError as e:
        print("Error reshaping 'indicator' column after filtering.")
        print(f"Expected grid size: ({num_x}, {num_y}) -> Total: {num_x * num_y}")
        print(f"Actual 'indicator' size after filtering: {len(filtered_data['indicator'])}")
        return

    # Set up the figure layout
    fig = plt.figure(figsize=(14, 12))
    fig.suptitle(f'Comparison of On-Top Ratio (OTR) for {molecule}', fontsize=16)

    # Left: Reference Image (Bibliography Results) - occupying the entire left side
    ax1 = fig.add_subplot(1, 2, 1)
    if reference_img is not None:
        ax1.imshow(reference_img)
        ax1.axis('off')
        ax1.set_title(f'Bibliography Results: {molecule}')
    else:
        ax1.text(0.5, 0.5, 'Reference Image Not Available', ha='center', va='center', fontsize=12)
        ax1.axis('off')
    
    # Right Top: Calculated Contour Plot with increased contour levels
    ax2 = fig.add_subplot(2, 2, 2)
    num_levels = 20  # Adjust this as needed
    contour = ax2.contourf(x_vals, y_vals, indicator_vals, cmap='rainbow', levels=num_levels)
    fig.colorbar(contour, ax=ax2, orientation='vertical')
    ax2.set_title(f'Calculated OTR Contour Plot: {molecule}')
    ax2.set_xlabel('z(Å)')
    ax2.set_ylabel('z(Å)')
    ax2.set_aspect('equal')  # Match aspect ratio
    
    # Right Bottom: Calculated Line Plot, filtered to x <= 1
    ax3 = fig.add_subplot(2, 2, 4)
    line_data = calculated_indicator[(calculated_indicator['y'] <= 0.000001) & (calculated_indicator['x'] <= limit)]
    ax3.plot(line_data['x'], line_data['indicator'], color='black')
    ax3.set_title(f'Calculated OTR Line Plot: {molecule}')
    ax3.set_xlabel('z(Å)')
    ax3.set_ylabel('R')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


In [26]:
plot_comparison(molecule.name,obtain_atomic_indicator(df), limit=1)

Error reshaping 'x', 'y', or 'indicator' columns. Ensure they can be reshaped to the detected grid size.
